# Several Guidance Signals

Again, this notebook was applied on train, validation and test split. 
Change (to val or test) according to your needs.

In [2]:
import pandas as pd   
import json

# 1. Events (GS1)

In [ ]:
file_path= "/Users/clementgillet/Desktop/4SUBMISSION/files/train_events.json"
events = pd.read_json(path_or_buf=file_path, lines=True)
events

In [ ]:
# We put all events in a list of lists while filtering out NaN and None results
# We also have to merge all 1215 columns into 1 
# We then assign this list to a column of the v2 of NarraSum called "event_mentions"
lsls = []
for i in range(109767):
    ls = []
    if i%1000==0:
        print(i)
    for j in range(1215):
        if events[j][i] != None:
            ls.append(events[j][i])
    lsls.append(ls)
    
#filtering NaN-entries

for i in range(109767):
    lsls[i] = [item for item in lsls[i] if not(pd.isnull(item)) == True]

# Here an example of the 1st input document's event mentions   

lsls[0]

In [ ]:
events = pd.DataFrame()
events["guidance"] = lsls
events.guidance

In [ ]:
# Prune out numerical features to keep just trigger word + event type --> GS1_base
for i in range(109767):
    for j in range(len(events.guidance[i])):
        del events.guidance[i][j]['sent_id']
        del events.guidance[i][j]['offset']
        del events.guidance[i][j]['id']
        del events.guidance[i][j]['type_id']
        

In [ ]:
with open('/Users/clementgillet/Desktop/GS1_base.json', 'w') as f:
    f.write(events.to_json(orient='records', lines=True))

# 2. COREFERENCE, TEMPORAL, CAUSAL, SUBEVENT (GS2)

In [ ]:
ls = []
for i in range(1,3):
    print(i)
    file_path= f"/Users/clementgillet/Desktop/train/{i}.jsonl"
    raw_rel = pd.read_json(path_or_buf=file_path, lines=True)
    ls.append(raw_rel)

In [ ]:
ls = []
for i in range(3,5):
    print(i)
    file_path= f"/Users/clementgillet/Desktop/train/{i}.jsonl"
    raw_rel = pd.read_json(path_or_buf=file_path, lines=True)
    ls.append(raw_rel)

In [ ]:
ls = []
for i in range(5,7):
    print(i)
    file_path= f"/Users/clementgillet/Desktop/train/{i}.jsonl"
    raw_rel = pd.read_json(path_or_buf=file_path, lines=True)
    ls.append(raw_rel)

In [ ]:
ls = []
for i in range(7,9):
    print(i)
    file_path= f"/Users/clementgillet/Desktop/train/{i}.jsonl"
    raw_rel = pd.read_json(path_or_buf=file_path, lines=True)
    ls.append(raw_rel)

In [ ]:
ls = []
for i in range(9,11):
    print(i)
    file_path= f"/Users/clementgillet/Desktop/train/{i}.jsonl"
    raw_rel = pd.read_json(path_or_buf=file_path, lines=True)
    ls.append(raw_rel)

In [ ]:
ls = []
for i in range(11,13):
    print(i)
    file_path= f"/Users/clementgillet/Desktop/train/{i}.jsonl"
    raw_rel = pd.read_json(path_or_buf=file_path, lines=True)
    ls.append(raw_rel)

In [ ]:
ls = []
for i in range(13,15):
    print(i)
    file_path= f"/Users/clementgillet/Desktop/train/{i}.jsonl"
    raw_rel = pd.read_json(path_or_buf=file_path, lines=True)
    ls.append(raw_rel)

In [ ]:
raw_relj = pd.concat(ls, ignore_index = True)

In [ ]:
raw_relj

In [ ]:
# clean all single coreferences
for i in range(1000):
    for ls in raw_relj.coreference:
        for elem in ls:
            if len(elem)==1:
                ls.remove(elem)
# there are coreference arrays that are up to 12 event_mentions long

In [ ]:
file_path1= "/Users/clementgillet/Desktop/4SUBMISSION/Guidance Signal/Train Split/GS1.json"
event_mentions_pre = pd.read_json(path_or_buf=file_path1, lines=True)

In [ ]:
event_mentions_pre

In [ ]:
EM = event_mentions_pre.loc[96032:109767].copy()
EM

In [ ]:
index = dropp.index[dropp.id =='73420b51d3f742c4d6f293deefae7bf0']
index
#dropp.document[20543]

In [ ]:
# take original NarraSum file and check what lines were discarded during 
# ERE-inference and drop them from event_mentions

file_path1= "/Users/clementgillet/Desktop/Master_Hub/datasets/NarraSum/train.json"
dropp = pd.read_json(path_or_buf=file_path1, lines=True)

In [ ]:
dropp.document[109763]

In [ ]:
dd = dropp.loc[96032:109767]

ls = list(set(dd.id) - set(raw_relj.id))
print(len(ls))
for idx, ide in enumerate(dropp.id):
    if ide in ls:
        EM.drop(idx, inplace=True)

In [ ]:
event_mentions = EM
event_mentions

In [ ]:
# 2 ids that are missing in RAW_GUIDANCE cause no event_mentions : ['01863e31835c64933f2ba529de61c0db', '55fd7c68cd5dd8ed47c8bd7e900e3c2f'] 
# indices : [5519, 1281]
# so we delete these 2 rows from the event_mentions dataframe so it fits to the dataframe above
#event_mentions.drop(5519, inplace=True)
#event_mentions.drop(1281, inplace=True)

In [ ]:
# COREFERENCE : replace all alphanumeric IDs with trigger word (+ event-mentions ?)
for ls1, ls2 in zip(raw_relj.coreference, event_mentions.guidance) :
    for elem in ls1:
        for idx, ID in enumerate(elem):
            for event in ls2:
                if ID == event['id']:
                    elem[idx] =  event['trigger-word']           

In [ ]:
# CAUSAL : replace all alphanumeric IDs with trigger word (+ event-mentions ?) !!!CAUSAL + PRECONDITION!!!
for enum, (ls1, ls2) in enumerate(zip(raw_relj.causal_relations, event_mentions.guidance)) :
    for cause_type in ls1:
        for pair in raw_relj.causal_relations[enum][cause_type]:
            for idx, ID in enumerate(pair):
                for event in ls2:
                    if ID == event['id']:
                        pair[idx] =  event['trigger-word'] 

In [ ]:
# snippet that sigularizes temporal relations --> this will spare a lot of time in proocessing
# for first id, remove all next pairs with this id (being first) and do a list of this 
# id with all others id that come aftter in time

In [ ]:
# TEMPORAL : replace all alphanumeric IDs with trigger word (+ event-mentions ?) !!!6 diff types of temp rel!!!
for enum, (ls1, ls2) in enumerate(zip(raw_relj.temporal_relations, event_mentions.guidance)) :
    print(enum)
    for temp_type in ls1:
        for pair in raw_relj.temporal_relations[enum][temp_type]:
            for idx, ID in enumerate(pair):
                for event in ls2:
                    if ID == event['id']:
                        pair[idx] =  event['trigger-word'] 

In [ ]:
# SUBEVENT : replace all alphanumeric IDs with trigger word (+ event-mentions ?)
for ls1, ls2 in zip(raw_relj.subevent_relations, event_mentions.guidance) :
    for elem in ls1:
        for idx, ID in enumerate(elem):
            for event in ls2:
                if ID == event['id']:
                    elem[idx] =  event['trigger-word']  

In [ ]:
raw_relj

In [ ]:
with open('/Users/clementgillet/Desktop/GS2_train_intermediary_7.json', 'w') as f:
    f.write(raw_relj.to_json(orient='records', lines=True))

# Textual Transformation + Merging Temporal Arrays

In [ ]:
# Re-arrange ERE-flagging with more linguistic cues ?

# Check if for one ERE-type, chronological order is applied
# what is the difference really between 
# rajouter des sentence position numbering in guidance ? tthis trigger word in sentence 4
# shorten temporal annotation

# Idées ??
# "trigger-word A (in sentence 3) and B (in sentence 10) refer to the same event" / A, B, C , ...
# "A causes B" and " A is a precondition to B"
# "A" overlaps with "B", "A begins on B"
# A happpens during that B is ongoing



In [ ]:
ls = []
for i in range(6,8):
    print(i)
    file_path= f"/Users/clementgillet/Desktop/GS2_train_intermediary_{i}.json"
    raw_rel = pd.read_json(path_or_buf=file_path, lines=True)
    ls.append(raw_rel)

In [ ]:
file_path= "/Users/clementgillet/Desktop/4SUBMISSION/Guidance Signal/Val Split /GS2_val_intermediary.json"
raw_rel = pd.read_json(path_or_buf=file_path, lines=True)

In [ ]:
raw_rel = pd.concat(ls, ignore_index = True)

In [ ]:
raw_relj = raw_rel
raw_relj

In [ ]:
# coreference (textual transformation)

for ls in raw_relj.coreference :
    for index, elem in enumerate(ls):
        string = elem[0] + " and "
        for idx in range(1, len(elem)):
            if idx==len(elem)-1:
                string += elem[idx] + " REFER to the same event "
                break
            string += elem[idx] + " and "
        ls[index]= string

In [ ]:
# causal (textual transformation)

for enum, ls in enumerate(raw_relj.causal_relations):
    for cause_type in ls:
        for idx, pair in enumerate(raw_relj.causal_relations[enum][cause_type]):
            string = pair[0] + " " + cause_type + "S " +  pair[1]
            ls[cause_type][idx] = string
                 

In [ ]:
# temporal (textual transformation + HUGE MERGING OPERATION)
metameta= []
for enum, ls in enumerate(raw_relj.temporal_relations):
    if enum%100==0:
        print(enum)
    listmeta = []
    for temp_type in ls:
        if len(raw_relj.temporal_relations[enum][temp_type])>0:
            if temp_type == "BEFORE":
                centralEvent = raw_relj.temporal_relations[enum][temp_type][0][0]
                mark = 0
                LIST = []
                for pair in raw_relj.temporal_relations[enum][temp_type]:
                    if pair[0] == centralEvent:
                        if mark == 1:
                            string += ", " + pair[1]
                        else:
                            string = centralEvent + f" {temp_type} " + pair[1]
                            mark = 1
                    else:
                        LIST.append(string)
                        centralEvent = pair[0]
                        mark = 0
            else:
                for pair in raw_relj.temporal_relations[enum][temp_type]:
                    string = pair[0] + f" {temp_type} " + pair[1]
                    LIST.append(string)
            listmeta.append(LIST)
            if enum%100==0:
                print(listmeta)
    metameta.append(listmeta)

In [ ]:
# subevent (textual transformation)

for enum, ls in enumerate(raw_relj.subevent_relations):
    for idx, pair in enumerate(ls):
            string = pair[1] + " HAPPENS DURING " +  pair[0]
            ls[idx] = string

In [ ]:
raw_relj["temporal_relations"]= metameta

In [ ]:
# keep only central event of all BEFORE arrays to finally obtain a chained chronology.
for idx in range(len(raw_relj.temporal_relations)):
    if len(raw_relj.temporal_relations[idx])>0:
        for i, elem in enumerate(raw_relj.temporal_relations[idx][0]):
            raw_relj.temporal_relations[idx][0][i] = raw_relj.temporal_relations[idx][0][i].split()[0]

In [ ]:
# add linguistic
for idx in range(len(raw_relj.temporal_relations)):
    if len(raw_relj.temporal_relations[idx])>0:
            raw_relj.temporal_relations[idx][0].insert(0, "The chronology of events is the following : ")
            

In [ ]:
for idx in range(len(raw_relj.causal_relations)):
    raw_relj.causal_relations[idx] = raw_relj.causal_relations[idx]['CAUSE'] + raw_relj.causal_relations[idx]['PRECONDITION']

In [ ]:
# On # rows :
# only 529 rows that contain subevent (804)(438) test: (102) val: (144)
# only 12065 row that contain coreference (18796)(10087) test: (2352) val: (2386)
# 30672 rows have temporal information (47050)(25447) test: (6076) val: (5917)
# 21354 rows have causal information (32885)(17852) test: (4095) val: (4091)

count = 0 
for elem in raw_relj.subevent_relations:
    if len(elem)>0:
        count += 1
        
print(count)

In [ ]:
raw_relj["guidance"] = raw_relj.temporal_relations + raw_relj.coreference + raw_relj.causal_relations + raw_relj.subevent_relations

In [ ]:
raw_relj.drop("coreference", axis=1, inplace=True)

In [ ]:
raw_relj.drop("temporal_relations", axis=1, inplace=True)

In [ ]:
raw_relj.drop("causal_relations", axis=1, inplace=True)

In [ ]:
raw_relj.drop("subevent_relations", axis=1, inplace=True)

In [ ]:
raw_relj.guidance[33]

In [ ]:
with open('/Users/clementgillet/Desktop/GS2_final_val.json', 'w') as f:
    f.write(raw_relj.to_json(orient='records', lines=True))

------------------------------------------------------------------------------------------------------------------

In [ ]:
file_path= "/Users/clementgillet/Desktop/4SUBMISSION/Guidance Signal/Test Split/GS2_final_test.json"
guidance = pd.read_json(path_or_buf=file_path, lines=True)

In [ ]:
guidance

In [ ]:
file_path= "/Users/clementgillet/Desktop/Master_Hub/datasets/NarraSum/train.json"
initial = pd.read_json(path_or_buf=file_path, lines=True)
initial1 = pd.read_json(path_or_buf=file_path, lines=True)

In [ ]:
initial

In [ ]:
ls = list(set(initial.id) - set(guidance.id))
print(len(ls))
for idx, ide in enumerate(initial1.id):
    if ide in ls:
        initial.drop(idx, inplace=True)

In [ ]:
# now the original NarraSum is truncated by 6212 examples (ca. 6% of dataset)

In [ ]:
# what other filter should we apply ? 
#Given that subevent information is scarce, if we filter all entries without subevent relation, 
# we lose too much of the dataset
# 309 elements are empty, remove them 

count = 0 
for idx, elem in enumerate(guidance.guidance):
    if len(elem)==0:
        count += 1
        guidance.drop(idx, inplace=True)
count

# no empty guidance anymore. That is our baseline !! 
# For every association of events, we are going to retrain BART original on a different subset ??? 
        

In [ ]:
with open('/Users/clementgillet/Desktop/GS2_final_val.json', 'w') as f:
    f.write(guidance.to_json(orient='records', lines=True))

In [ ]:
with open('/Users/clementgillet/Desktop/val.json', 'w') as f:
    f.write(initial.to_json(orient='records', lines=True))

In [ ]:
file_path= "/Users/clementgillet/Desktop/4SUBMISSION/Guidance_Signal/GS1_guid/test.json"
initial = pd.read_json(path_or_buf=file_path, lines=True)

In [ ]:
file_path1= "/Users/clementgillet/Desktop/Master_Hub/datasets/NarraSum/validation.json"
realinit = pd.read_json(path_or_buf=file_path1, lines=True)

In [ ]:
file_path= "/Users/clementgillet/Desktop/INITIAL_filtered/GS2/train.json"
initial1 = pd.read_json(path_or_buf=file_path, lines=True)

In [ ]:
ls = list(set(initial1.id) - set(initial.id))
print(len(ls))
for idx, ide in enumerate(initial1.id):
    if ide in ls:
        initial2.drop(idx, inplace=True)

In [ ]:
initial["id"] = ""
initial=initial.rename(columns = {'guidance':'document'})
initial["summary"] = ""
initial.summary[0]

In [ ]:
initial = initial[['id', 'document', 'summary']]

In [ ]:
initial.document

In [ ]:
for idx in range(len(initial.document)):
    initial.document[idx] = ' '.join(str(x) for x in initial.document[idx])

In [ ]:
for idx in range(len(initial.document)):
    initial.document[idx] = initial.document[idx].replace("[", "")
    initial.document[idx] = initial.guidance[idx].replace("]", "")
    initial.document[idx] = initial.guidance[idx].replace("'", "")

In [ ]:
initial.document[503]

In [ ]:
with open('/Users/clementgillet/Desktop/test.json', 'w') as f:
    f.write(initial.to_json(orient='records', lines=True))

# 3.  COREFERENCE, TEMPORAL, CAUSAL (GS3)

Ablate subevent from initial signal

In [411]:
file_path= "/Users/clementgillet/Desktop/4SUBMISSION/Guidance_Signal/Test Split/GS3.json"
GS = pd.read_json(path_or_buf=file_path, lines=True)

In [412]:
file_path1= "/Users/clementgillet/Desktop/Master_Hub/datasets/NarraSum/test.json"
DS = pd.read_json(path_or_buf=file_path1, lines=True)
DS1 = pd.read_json(path_or_buf=file_path1, lines=True)

In [353]:
for idx in range(len(GS2_1.guidance)):
    if len(GS2_1.guidance[idx])==0:
        GS2_1.drop(idx, inplace=True)
        idx -= 1

In [232]:
GS2_1.reset_index(inplace=True)

In [238]:
GS2_1

,level_0,index,id,guidance
6,6,6,97050b204c9aee0835d73b60ee0253f9,"[fly and leave REFER to the same event , stole..."
9,9,9,5a2ccadd45fd23461b6355201c2eae5e,[Fariba and killings REFER to the same event ]
10,10,10,6b447508813508a907c1c612a3d3af5f,[exchanging and shots and shooting and exchang...
16,16,16,6db180303c303e6f3f4cdd096191e922,[killed and killed REFER to the same event ]
17,17,17,3e539c180ffba280b3b9e0f4b854bfe9,[hitting and hit REFER to the same event ]
...,...,...,...,...
30671,30671,30771,862a4574f0e2f34a58c18f1dbac133a6,[trial and trial REFER to the same event ]
30673,30673,30773,70c24614aef784bcc9154b0eabb34fd8,[leading and investigation REFER to the same e...
30674,30674,30774,53f1848824e82fcc874a36b3e5397799,[record and record REFER to the same event ]
30675,30675,30775,885ea3d38a5152956ade8acc912895ba,"[use and declines REFER to the same event , vo..."


In [352]:
for i in range(len(GS2_1.guidance)):
    GS2_1.guidance[i] = [elem for elem in GS2_1.guidance[i] if "HAPPENS DURING" not in elem]
    GS2_1.guidance[i] = [elem for elem in GS2_1.guidance[i] if "CAUSES" not in elem]
    GS2_1.guidance[i] = [elem for elem in GS2_1.guidance[i] if "PRECONDITIONS" not in elem]
    if len(GS2_1.guidance[i]) > 0:
        del GS2_1.guidance[i][0]
        

In [354]:
initial = GS2_1
initial

,id,guidance
4,92dd585fbe6fb2da29f98eee29d2db9d,[battle and battle REFER to the same event ]
6,71a05de42d306fe6fdc1deb6ef9f4b69,"[running and ran REFER to the same event , wed..."
10,38d8be082aa2e21e891cd8bebc4fd917,[reception and reception REFER to the same eve...
13,5facb7742f1b4d03f95c39980a6d42e2,"[save and save REFER to the same event , disas..."
14,ef2039381659b5bf041641dad3ab1373,"[wedding and wedding REFER to the same event ,..."
...,...,...
6066,94b840f69d37c417ac4fe1a5731777ee,[killed and killed and killed REFER to the sam...
6069,3c81c5ed665a5837a16fde26da77dd98,[marriage and marriage REFER to the same event...
6073,a457183e2a877fe487161130739ea2be,[interrogation and interrogation and interroga...
6076,d1d63bd8d81e73fb0870183827637dc2,[survey and survey and survey REFER to the sam...


In [355]:
initial.reset_index(inplace=True)

In [356]:
initial=initial.rename(columns = {'guidance':'document'})
initial["summary"] = ""

In [357]:
initial = initial[['id', 'document', 'summary']]

In [358]:
initial

,id,document,summary
0,92dd585fbe6fb2da29f98eee29d2db9d,[battle and battle REFER to the same event ],
1,71a05de42d306fe6fdc1deb6ef9f4b69,"[running and ran REFER to the same event , wed...",
2,38d8be082aa2e21e891cd8bebc4fd917,[reception and reception REFER to the same eve...,
3,5facb7742f1b4d03f95c39980a6d42e2,"[save and save REFER to the same event , disas...",
4,ef2039381659b5bf041641dad3ab1373,"[wedding and wedding REFER to the same event ,...",
...,...,...,...
2351,94b840f69d37c417ac4fe1a5731777ee,[killed and killed and killed REFER to the sam...,
2352,3c81c5ed665a5837a16fde26da77dd98,[marriage and marriage REFER to the same event...,
2353,a457183e2a877fe487161130739ea2be,[interrogation and interrogation and interroga...,
2354,d1d63bd8d81e73fb0870183827637dc2,[survey and survey and survey REFER to the sam...,


In [359]:
for idx in range(len(initial.document)):
    initial.document[idx] = ''.join(str(x) for x in initial.document[idx])

In [360]:
for idx in range(len(initial.document)):
    initial.document[idx] = initial.document[idx].replace("[", "")
    initial.document[idx] = initial.document[idx].replace("]", "")
    initial.document[idx] = initial.document[idx].replace("'", "")

In [361]:
initial

,id,document,summary
0,92dd585fbe6fb2da29f98eee29d2db9d,battle and battle REFER to the same event,
1,71a05de42d306fe6fdc1deb6ef9f4b69,running and ran REFER to the same event weddin...,
2,38d8be082aa2e21e891cd8bebc4fd917,reception and reception REFER to the same event,
3,5facb7742f1b4d03f95c39980a6d42e2,save and save REFER to the same event disaster...,
4,ef2039381659b5bf041641dad3ab1373,wedding and wedding REFER to the same event dr...,
...,...,...,...
2351,94b840f69d37c417ac4fe1a5731777ee,killed and killed and killed REFER to the same...,
2352,3c81c5ed665a5837a16fde26da77dd98,marriage and marriage REFER to the same event ...,
2353,a457183e2a877fe487161130739ea2be,interrogation and interrogation and interrogat...,
2354,d1d63bd8d81e73fb0870183827637dc2,survey and survey and survey REFER to the same...,


In [284]:
ls1 = [initial]

In [334]:
ls1.append(initial)

In [335]:
ls1

[                                     id  \
 0      97050b204c9aee0835d73b60ee0253f9   
 1      5a2ccadd45fd23461b6355201c2eae5e   
 2      6b447508813508a907c1c612a3d3af5f   
 3      6db180303c303e6f3f4cdd096191e922   
 4      3e539c180ffba280b3b9e0f4b854bfe9   
 ...                                 ...   
 12093  862a4574f0e2f34a58c18f1dbac133a6   
 12094  70c24614aef784bcc9154b0eabb34fd8   
 12095  53f1848824e82fcc874a36b3e5397799   
 12096  885ea3d38a5152956ade8acc912895ba   
 12097  bf025af53cb59bcaebe4d941fdb16891   
 
                                                 document summary  
 0      fly and leave REFER to the same event  stole a...          
 1           Fariba and killings REFER to the same event           
 2      exchanging and shots and shooting and exchange...          
 3             killed and killed REFER to the same event           
 4               hitting and hit REFER to the same event           
 ...                                                  ...     

In [362]:
initial

,id,document,summary
0,92dd585fbe6fb2da29f98eee29d2db9d,battle and battle REFER to the same event,
1,71a05de42d306fe6fdc1deb6ef9f4b69,running and ran REFER to the same event weddin...,
2,38d8be082aa2e21e891cd8bebc4fd917,reception and reception REFER to the same event,
3,5facb7742f1b4d03f95c39980a6d42e2,save and save REFER to the same event disaster...,
4,ef2039381659b5bf041641dad3ab1373,wedding and wedding REFER to the same event dr...,
...,...,...,...
2351,94b840f69d37c417ac4fe1a5731777ee,killed and killed and killed REFER to the same...,
2352,3c81c5ed665a5837a16fde26da77dd98,marriage and marriage REFER to the same event ...,
2353,a457183e2a877fe487161130739ea2be,interrogation and interrogation and interrogat...,
2354,d1d63bd8d81e73fb0870183827637dc2,survey and survey and survey REFER to the same...,


In [413]:
ls = list(set(DS.id) - set(GS.id))
print(len(ls))
for idx, ide in enumerate(DS.id):
    if ide in ls:
        DS1.drop(idx, inplace=True)

38


In [409]:
DS1

,id,document,summary
0,195e3a541827dc23f57638e46e055700,After boxer Joe Palooka bests South American f...,"Joe Palooka kayos Cardona, but the latter prot..."
1,45b391802075b9b7dabad6e7c9342850,The opening sequence is different from many ot...,"Quagmire returns from a vacation in Florida , ..."
2,d3d738675ef0b52f03ffee3440fcd2f9,The Masher played by Charlie Chaplin fights fo...,Charlie and a rival vie for the favors of thei...
3,699a09647d35739ecc1b0aa203e199cf,The Federation starship Enterprise is ordered ...,"To Troi's dismay, her planet's delegate to the..."
4,92dd585fbe6fb2da29f98eee29d2db9d,"The town of Barrow, Alaska is preparing for it...","In Barrow, Alaska, seventy percent of the popu..."
...,...,...,...
6116,e39568c4f47f08bfca7ba40b99992cc3,"In Pottsboro , North Texas, Charlie finds hers...",Rachel awakens after three days unconsciousnes...
6117,894167d1e82aea9324c96bb0000044fe,A wife waits for her brawling sailor ex-husban...,A Wrestler and his ex-Wife (Noah Young and Fay...
6118,6ea99e87a13dbc5f4a71c0c074d87c08,A convention of white supremacists led by Russ...,Some of the hotel's guests who are part of a c...
6119,18ce218b68017bb0d616270e8d134ba0,1944 in the small town of Tigreville on the co...,With his dutiful but unimaginative wife Suzann...


In [410]:
with open('/Users/clementgillet/Desktop/test.json', 'w') as f:
    f.write(DS1.to_json(orient='records', lines=True))

# 4.  COREFERENCE, TEMPORAL (GS4)
Ablate subevent, causal from initial signal

In [12]:
file_path= "/Users/clementgillet/Desktop/4SUBMISSION/Guidance_Signal/GS5_guid/validation.json"
GS = pd.read_json(path_or_buf=file_path, lines=True)

In [13]:
GS

,id,document,summary
0,1ec7843deddb3947133d9f96880bd198,plan and plan REFER to the same event wedding ...,
1,739cafa77a0a4e6117577a6aab86c775,Business and party and party and party REFER t...,
2,c0dbea44a72daf8b4ae90faafb3525ae,murder and death and murder and death REFER to...,
3,b86ab81bdef9f9b57205164b3cf52185,taken and took REFER to the same event commit ...,
4,56ade5296ae36d362a76626af72e2b7e,meeting and meeting and meeting REFER to the s...,
...,...,...,...
2396,f91485f6cac7c3f237ba5c3a4cbfd433,insists and insists REFER to the same event,
2397,af608759f7fee0826e7e420b4230be0e,party and party REFER to the same event kills ...,
2398,4165b907ebfebca22892604c5304a54f,swapped and swap REFER to the same event,
2399,d6484b2ea65ace5617223add1d0b2bf7,party and party REFER to the same event compet...,


In [7]:
with open('/Users/clementgillet/Desktop/4SUBMISSION/Guidance_Signal/GS5_guid/test.json', 'w') as f:
    f.write(GS.to_json(orient='records', lines=True))

# 5.  COREFERENCE (GS5)
Ablate subevent, causal and temporal from initial signal